In [ ]:
import polars as pl
import os
from util import pdf_to_text, best_fuzzy_match

In [ ]:
ms_cases = [c.replace(".pdf", "") for c in os.listdir("Cases/ManuallyScrapedCases/")]
df = pl.read_csv("./Cases/WestLawNotMatch.csv")
df.head()

Title,Court Line,Citation,OpinionURL,CourtListenerCaseName,Match
str,str,str,str,str,i64
"""O'Neil v. Rata…","""United States …","""563 F.Supp.3d …",null,null,0
"""Furie v. Infow…","""United States …","""401 F.Supp.3d …",null,null,0
"""Advanta-STAR A…","""United States …","""672 F.Supp.3d …",null,null,0
"""4DD Holdings, …","""United States …","""143 Fed.Cl. 11…",null,null,0
"""Apple Inc. v. …","""United States …","""510 F.Supp.3d …",null,null,0


In [ ]:
df = df.with_columns(
    pl.col("Title").map_elements(lambda x: best_fuzzy_match(x, choices=ms_cases)[0]+".pdf").alias("Matched"),
    pl.col("Title").map_elements(lambda x: best_fuzzy_match(x, choices=ms_cases)[1]).alias("MatchedScore")
).sort("MatchedScore")

In [24]:
df = df.filter( pl.col("MatchedScore") > 49).with_columns(
    pl.col("Matched").map_elements(lambda x: pdf_to_text("./Cases/ManuallyScrapedCases/" + x)).alias("Opinion")
)

In [31]:
df.write_csv("./Cases/WLNM_Opinion.csv")